In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import kagglehub

/Users/trevorchartier/Documents/Career/Independent Study/PhishNet/phishy_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Data

In [2]:
# Download latest version of data
path = kagglehub.dataset_download("naserabdullahalam/phishing-email-dataset")
print("Path to dataset files:", path)
data = pd.read_csv(path +"/phishing_email.csv")
data.head()

Path to dataset files: /Users/trevorchartier/.cache/kagglehub/datasets/naserabdullahalam/phishing-email-dataset/versions/1


,text_combined,label
0,hpl nom may 25 2001 see attached file hplno 52...,0
1,nom actual vols 24 th forwarded sabrae zajac h...,0
2,enron actuals march 30 april 1 201 estimated a...,0
3,hpl nom may 30 2001 see attached file hplno 53...,0
4,hpl nom june 1 2001 see attached file hplno 60...,0


In [3]:
print("Shape of Data:", data.shape)
print("Number of phishing instances: ", sum(data['label'] == 1))
print("Number of legitimate instances: ", sum(data['label'] == 0))

print("\nAny Missing Values: ", pd.isna(data).any().any())

Shape of Data: (82486, 2)
Number of phishing instances:  42891
Number of legitimate instances:  39595

Any Missing Values:  False


## Splitting Data

In [4]:
train_and_val, test_df = train_test_split(data, test_size=0.2)
train_df, val_df = train_test_split(train_and_val, test_size=0.25)

## PyTorch Compatible EmailDataset Class

In [5]:
class EmailDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length = 512):
        self.dataset = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        text = self.dataset.iloc[idx]['text_combined']
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {'input_ids': encoding['input_ids'].squeeze(), 
                'attention_mask': encoding['attention_mask'].squeeze(),
                'labels': torch.tensor(self.dataset.iloc[idx]['label'])}

## Tokenization


In [6]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

train_dataset = EmailDataset(train_df, tokenizer)
val_dataset = EmailDataset(val_df, tokenizer)
test_dataset = EmailDataset(test_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-4)
num_epochs = 3

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the Model

In [12]:
((len(train_loader)/2) * 9.5) / 60 / 24

5.102951388888889

In [7]:
device = 'cpu'
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch_inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")
        

Epoch 1, Loss: 0.0004690337658853167
Epoch 1, Loss: 0.0009124047847123938


KeyboardInterrupt: 